# PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending per student actually (\$645-675) underperformed compared to schools with smaller budgets (<\$585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 
---

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, 
                                how="left", on=["school_name", "school_name"])

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [2]:
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [3]:
total_schools = school_data_complete['School ID'].nunique()
total_students = school_data_complete['Student ID'].nunique()
total_budget = school_data_complete['budget'].nunique()
by_school_df = school_data_complete.groupby(['School ID']).mean()
total_budget = by_school_df['budget'].sum()
math_avg = school_data_complete['math_score'].mean()
reading_avg = school_data_complete['reading_score'].mean()
overall_passing_rate = (math_avg + reading_avg)/2
math_students_70 = school_data_complete[(school_data_complete.math_score >= 70)]\
    ['Student ID'].count()/ total_students * 100
reading_students_70 = school_data_complete[(school_data_complete.
                                            reading_score >= 70)]['Student ID']\
                                            .count()/ total_students * 100
df = pd.DataFrame({'Total Schools':[total_schools], 
                   'Total Students': [total_students],
                   'Total Budget': [total_budget], 'Math Avg':[math_avg],
                   'Reading Avg':[reading_avg], 
                   'Overall Passing Rate': [overall_passing_rate],
                   '% of Students above 70 Math': [math_students_70],
                   '% of Students above 70 Reading': [reading_students_70]},
                  index=None)
df['Total Budget'] = df['Total Budget'].map('${:,.0f}'.format)
df['Total Students'] = df['Total Students'].map('{:,.0f}'.format)
df['Math Avg'] = df['Math Avg'].map('{:,.0f}%'.format)
df['Reading Avg'] = df['Reading Avg'].map('{:,.0f}%'.format)
df['Overall Passing Rate'] = df['Overall Passing Rate'].map('{:,.0f}%'.format)
df['% of Students above 70 Math'] = df['% of Students above 70 Math']\
                                    .map('{:,.0f}%'.format) 
df['% of Students above 70 Reading'] = df['% of Students above 70 Reading']\
                                    .map('{:,.0f}%'.format) 
df

,Total Schools,Total Students,Total Budget,Math Avg,Reading Avg,Overall Passing Rate,% of Students above 70 Math,% of Students above 70 Reading
0,15,"39,170","$24,649,428",79%,82%,80%,75%,86%


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

In [4]:
overview_df_raw = pd.DataFrame(school_data_complete.groupby(['school_name'], 
            as_index=True).agg({'type': 'max', 'student_name':'count',
                                'budget': 'max', 'math_score': 'mean', 
                                'reading_score': 'mean'}))

passing_math_school_df_raw = pd.DataFrame(school_data_complete.loc
                        [school_data_complete['math_score'] > 69, :]
                        .groupby('school_name')['math_score'].count())
passing_reading_school_df_raw = pd.DataFrame(school_data_complete.loc
                            [school_data_complete['reading_score'] > 69, :]
                            .groupby('school_name')['reading_score'].count())

overview_df_raw = passing_math_school_df_raw.merge(overview_df_raw,on='school_name')
overview_df_raw = pd.DataFrame(passing_reading_school_df_raw.
                           merge(overview_df_raw,on='school_name'))

overview_df_raw.reset_index(inplace=True)
overview_df_raw = overview_df_raw.rename(columns={'school_name': 'Name', 
                            'reading_score_x': 'reading_raw', 
                            'math_score_x': 'math_raw', 'type': 'School Type', 
                            'student_name': 'Total Students', 
                            'budget': 'Total School Budget',
                            'math_score_y': 'Avg Math Score', 'reading_score_y': 
                            'Avg Reading Score'})

overview_df_raw['Per Student Budget']= (overview_df_raw['Total School Budget']/ 
                                    overview_df_raw['Total Students'])
overview_df_raw['% Passing Math']= (overview_df_raw['math_raw']/ 
                                overview_df_raw['Total Students'])* 100
overview_df_raw['% Passing Reading']= (overview_df_raw['reading_raw']/ 
                                overview_df_raw['Total Students'])* 100
overview_df_raw['% Overall Passing Rate']= (overview_df_raw['% Passing Math']+ 
                                        overview_df_raw['% Passing Reading'])/2
overview_df_raw = overview_df_raw[['Name', 'School Type','Total Students',
                           'Total School Budget','Per Student Budget', 
                            'Avg Math Score', 'Avg Reading Score', 
                           '% Passing Math', '% Passing Reading', 
                           '% Overall Passing Rate']]

overview_df_raw.set_index('Name', inplace=True)
del overview_df_raw.index.name
overview_df_raw

,School Type,Total Students,Total School Budget,Per Student Budget,Avg Math Score,Avg Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Bailey High School,District,4976,3124928,628.0,77.048432,81.033963,66.680064,81.933280,74.306672
Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,94.133477,97.039828,95.586652
Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.988471,80.739234,73.363852
Ford High School,District,2739,1763916,644.0,77.102592,80.746258,68.309602,79.299014,73.804308
Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.392371,97.138965,95.265668
Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,66.752967,80.862999,73.807983
Holden High School,Charter,427,248087,581.0,83.803279,83.814988,92.505855,96.252927,94.379391
Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,66.057551,81.222432,73.639992
Pena High School,Charter,962,585858,609.0,83.839917,84.044699,94.594595,95.945946,95.270270


## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [5]:
top_schools = overview_df_raw.sort_values('% Overall Passing Rate', 
                                      ascending=False).iloc[0:5,]
top_schools

,School Type,Total Students,Total School Budget,Per Student Budget,Avg Math Score,Avg Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,94.133477,97.039828,95.586652
Thomas High School,Charter,1635,1043130,638.0,83.418349,83.848930,93.272171,97.308869,95.290520
Pena High School,Charter,962,585858,609.0,83.839917,84.044699,94.594595,95.945946,95.270270
Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.392371,97.138965,95.265668
Wilson High School,Charter,2283,1319574,578.0,83.274201,83.989488,93.867718,96.539641,95.203679


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [6]:
bottom_schools = overview_df_raw.sort_values('% Overall Passing Rate', 
                                      ascending=True).iloc[0:5,]
bottom_schools

,School Type,Total Students,Total School Budget,Per Student Budget,Avg Math Score,Avg Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Rodriguez High School,District,3999,2547363,637.0,76.842711,80.744686,66.366592,80.220055,73.293323
Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.988471,80.739234,73.363852
Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,66.057551,81.222432,73.639992
Ford High School,District,2739,1763916,644.0,77.102592,80.746258,68.309602,79.299014,73.804308


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

In [7]:
ninth_m = school_data_complete.loc[school_data_complete['grade'] == '9th',]\
        .groupby('school_name')['math_score'].mean()
tenth_m = school_data_complete.loc[school_data_complete['grade'] == '10th',]\
        .groupby('school_name')['math_score'].mean()
eleventh_m = school_data_complete.loc[school_data_complete['grade'] == '11th',]\
        .groupby('school_name')['math_score'].mean()
twelfth_m = school_data_complete.loc[school_data_complete['grade'] == '12th',]\
        .groupby('school_name')['math_score'].mean()

In [8]:
from functools import reduce
math_grades = [ninth_m, tenth_m, eleventh_m, twelfth_m]
avg_math_by_grade = reduce(lambda  left,right: pd.merge(left,right,
                    on=['school_name'],how='outer'), math_grades)
avg_math_by_grade.columns = ['9th', '10th', '11th', '12th']
avg_math_by_grade['9th'] = avg_math_by_grade['9th'].map('{:,.2f}%'.format)
avg_math_by_grade['10th'] = avg_math_by_grade['10th'].map('{:,.2f}%'.format)
avg_math_by_grade['11th'] = avg_math_by_grade['11th'].map('{:,.2f}%'.format)
avg_math_by_grade['12th'] = avg_math_by_grade['12th'].map('{:,.2f}%'.format)
avg_math_by_grade

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.08%,77.00%,77.52%,76.49%
Cabrera High School,83.09%,83.15%,82.77%,83.28%
Figueroa High School,76.40%,76.54%,76.88%,77.15%
Ford High School,77.36%,77.67%,76.92%,76.18%
Griffin High School,82.04%,84.23%,83.84%,83.36%
Hernandez High School,77.44%,77.34%,77.14%,77.19%
Holden High School,83.79%,83.43%,85.00%,82.86%
Huang High School,77.03%,75.91%,76.45%,77.23%
Johnson High School,77.19%,76.69%,77.49%,76.86%


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [9]:
ninth_r = school_data_complete.loc[school_data_complete['grade'] == '9th',]\
        .groupby('school_name')['reading_score'].mean()
tenth_r = school_data_complete.loc[school_data_complete['grade'] == '10th',]\
        .groupby('school_name')['reading_score'].mean()
eleventh_r = school_data_complete.loc[school_data_complete['grade'] == '11th',]\
        .groupby('school_name')['reading_score'].mean()
twelfth_r = school_data_complete.loc[school_data_complete['grade'] == '12th',]\
        .groupby('school_name')['reading_score'].mean()

In [10]:
reading_grades = [ninth_r, tenth_r, eleventh_r, twelfth_r]
avg_reading_by_grade = reduce(lambda  left,right: pd.merge(left,right,
                    on=['school_name'],how='outer'), reading_grades)
avg_reading_by_grade.columns = ['9th', '10th', '11th', '12th']
avg_reading_by_grade['9th'] = avg_reading_by_grade['9th'].map('{:,.2f}%'.format)
avg_reading_by_grade['10th'] = avg_reading_by_grade['10th'].map('{:,.2f}%'.format)
avg_reading_by_grade['11th'] = avg_reading_by_grade['11th'].map('{:,.2f}%'.format)
avg_reading_by_grade['12th'] = avg_reading_by_grade['12th'].map('{:,.2f}%'.format)
avg_reading_by_grade

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.30%,80.91%,80.95%,80.91%
Cabrera High School,83.68%,84.25%,83.79%,84.29%
Figueroa High School,81.20%,81.41%,80.64%,81.38%
Ford High School,80.63%,81.26%,80.40%,80.66%
Griffin High School,83.37%,83.71%,84.29%,84.01%
Hernandez High School,80.87%,80.66%,81.40%,80.86%
Holden High School,83.68%,83.32%,83.82%,84.70%
Huang High School,81.29%,81.51%,81.42%,80.31%
Johnson High School,81.26%,80.77%,80.62%,81.23%


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [23]:
# Sample bins.
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

In [24]:
overview_df_raw['Budget Bins']= pd.cut(overview_df_raw['Per Student Budget'],
                                   spending_bins, labels=group_names)
Budget_bins_df = pd.DataFrame(overview_df_raw.groupby(['Budget Bins'], 
            as_index=True).agg({'Avg Math Score': 'mean', 'Avg Reading Score'\
                                :'mean','% Passing Math': 'mean', 
                                '% Passing Reading': 'mean'}))
Budget_bins_df['Overall % Passing'] = (Budget_bins_df['% Passing Math']+
                                        Budget_bins_df['% Passing Reading'])/2
Budget_bins_df

,Avg Math Score,Avg Reading Score,% Passing Math,% Passing Reading,Overall % Passing
Budget Bins,,,,,
<$585,83.455399,83.933814,93.460096,96.610877,95.035486
$585-615,83.599686,83.885211,94.230858,95.900287,95.065572
$615-645,79.079225,81.891436,75.668212,86.106569,80.887391
$645-675,76.997210,81.027843,66.164813,81.133951,73.649382


## Scores by School Size

* Perform the same operations as above, based on school size.

In [25]:
# Sample bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [26]:
overview_df_raw['Student Size Bins']= pd.cut(overview_df_raw['Total Students'],
                                   size_bins, labels=group_names)
size_bins_df = pd.DataFrame(overview_df_raw.groupby(['Student Size Bins'], 
            as_index=True).agg({'Avg Math Score': 'mean', 'Avg Reading Score'\
                                :'mean','% Passing Math': 'mean', 
                                '% Passing Reading': 'mean'}))
size_bins_df['Overall % Passing'] = (size_bins_df['% Passing Math']+
                                        size_bins_df['% Passing Reading'])/2
size_bins_df

,Avg Math Score,Avg Reading Score,% Passing Math,% Passing Reading,Overall % Passing
Student Size Bins,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,94.824831
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,95.195187
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,76.364998


## Scores by School Type

* Perform the same operations as above, based on school type.

In [27]:
type_bins_df = pd.DataFrame(overview_df_raw.groupby(['School Type'], 
            as_index=True).agg({'Avg Math Score': 'mean', 'Avg Reading Score'\
                                :'mean','% Passing Math': 'mean', 
                                '% Passing Reading': 'mean'}))
type_bins_df['Overall % Passing'] = (type_bins_df['% Passing Math']+
                                        type_bins_df['% Passing Reading'])/2
type_bins_df

,Avg Math Score,Avg Reading Score,% Passing Math,% Passing Reading,Overall % Passing
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,95.103660
District,76.956733,80.966636,66.548453,80.799062,73.673757
